In [1]:
# Must libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import data
imdb_data=pd.read_csv('data/IMDB_reviews.csv')

# print(imdb_data.shape,'\n')
# print(imdb_data.head(10),'\n')
# print(imdb_data['sentiment'].value_counts())

### Step 1 - Preprocessing Text to tokens

In [4]:
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [21]:
# %time
# # Pre-processing text data

# def preprocessing(text):
#     soup = BeautifulSoup(text, "html.parser")
#     text = soup.get_text()     # Step 1 - Remove html tags
#     pattern = r'[^a-zA-z0-9\s]' 
#     text = re.sub(pattern,'',text)   # Step 2 - Remove punctuation
#     text = text.lower()     # Step 3 - Make text lowercase
#     tokenizer=ToktokTokenizer()
#     tokens = tokenizer.tokenize(text)    # Step 4 - Tokenizing
#     lemma = WordNetLemmatizer()          # Step 5 - Lemmatizing
#     stopword_list=nltk.corpus.stopwords.words('english')
#     stop=set(stopwords.words('english'))
#     filtered_tokens = [lemma.lemmatize(word).strip() for word in tokens if word not in stopword_list]    # Step 6 - Removing stopwords
#     text = TreebankWordDetokenizer().detokenize(filtered_tokens)
#     return text

# #Apply function on review column
# imdb_data['review'] = imdb_data['review'].apply(preprocessing)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11 µs


In [23]:
# imdb_data.to_csv('data/IMDB_reviews_preprocessed.csv',index=False)

### Step 2 - Tokens to features and train-test split

In [6]:
imdb_data = pd.read_csv('data/IMDB_reviews_preprocessed.csv')
imdb_data.head()

,review,sentiment
0,one reviewer mentioned watching 1 oz episode y...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter matteis love time money visually stunni...,positive


In [7]:
norm_train_reviews=imdb_data.review[:40000]
norm_test_reviews=imdb_data.review[40000:]

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

def dummy(doc):
    return doc

# Count vectorizer for bag of words
cv = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
#        ngram_range=(1,3)
    )  

#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
cv_test_reviews=cv.transform(norm_test_reviews)

# Tf-idf vectorizer for bag of words
tfidf = TfidfVectorizer(
        tokenizer=dummy,
        preprocessor=dummy
    )  
tfidf_train_reviews = tfidf.fit_transform(norm_train_reviews)
tfidf_test_reviews = tfidf.transform(norm_test_reviews)


In [55]:
#labeling the sentiment data
lb=LabelBinarizer()

#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

#Spliting the sentiment data
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]

(50000, 1)


### Step 3 - Fit and predict

In [59]:
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)

#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_reviews,train_sentiments)
print(lr_bow)

#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_reviews)
lr_bow_predict
# lr.coef_

LogisticRegression(C=1, max_iter=500, random_state=42)


array([1, 0, 1, ..., 0, 0, 0])

In [49]:
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tfidf_train_reviews,train_sentiments)
print(lr_tfidf)

#Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tfidf_test_reviews)

LogisticRegression(C=1, max_iter=500, random_state=42)


### Step 4 - Metrics Evaluation

In [60]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(test_sentiments,lr_bow_predict)
print("lr_bow_accuracy :",lr_bow_score)

#Classification report for bag of words 
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print('\nlr_bow_classification_report:\n', lr_bow_report)

#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print('lr_bow_confusion_matrix:\n', cm_bow)

lr_bow_accuracy : 0.6173

lr_bow_classification_report:
               precision    recall  f1-score   support

    Positive       0.62      0.62      0.62      4993
    Negative       0.62      0.62      0.62      5007

    accuracy                           0.62     10000
   macro avg       0.62      0.62      0.62     10000
weighted avg       0.62      0.62      0.62     10000

lr_bow_confusion_matrix:
 [[3100 1907]
 [1920 3073]]


In [25]:
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_accuracy :",lr_tfidf_score)

#Classification report for tfidf features
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print('\nlr_tfidf_classification_report:\n', lr_tfidf_report)

#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print('lr_tfidf_confusion_matrix:\n', cm_tfidf)

lr_tfidf_accuracy : 0.616

lr_tfidf_classification_report:
               precision    recall  f1-score   support

    Positive       0.62      0.61      0.61      4993
    Negative       0.61      0.63      0.62      5007

    accuracy                           0.62     10000
   macro avg       0.62      0.62      0.62     10000
weighted avg       0.62      0.62      0.62     10000

lr_tfidf_confusion_matrix:
 [[3138 1869]
 [1971 3022]]


In [67]:
#training the model
mnb=MultinomialNB()

#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)

#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)

MultinomialNB()
[1 0 0 ... 0 0 0]


In [68]:
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tfidf_train_reviews,train_sentiments)
print(mnb_tfidf)

#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tfidf_test_reviews)
print(mnb_tfidf_predict)

MultinomialNB()
[0 0 1 ... 1 0 0]


In [69]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

#Classification report for bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)

#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
print(cm_bow)

mnb_bow_score : 0.6084
              precision    recall  f1-score   support

    Positive       0.60      0.63      0.62      4993
    Negative       0.61      0.59      0.60      5007

    accuracy                           0.61     10000
   macro avg       0.61      0.61      0.61     10000
weighted avg       0.61      0.61      0.61     10000

[[2945 2062]
 [1854 3139]]


In [70]:
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
print(cm_tfidf)

mnb_tfidf_score : 0.6135
              precision    recall  f1-score   support

    Positive       0.61      0.63      0.62      4993
    Negative       0.62      0.60      0.61      5007

    accuracy                           0.61     10000
   macro avg       0.61      0.61      0.61     10000
weighted avg       0.61      0.61      0.61     10000

[[3005 2002]
 [1863 3130]]
